# Generic Types Examples - Meaningful vs Trivial Usage

## 📋 Overview

This notebook demonstrates meaningful vs trivial usage of generic types, methods, and constraints in real-world scenarios.

**Key Learning Goals:**
- Understand when generics add real business value
- Learn proper constraint usage for type safety
- See examples of reusable generic components
- Understand performance benefits of generics

## ✅ Meaningful Usage Examples

### 🎯 Example 1: Repository Pattern with Generic Constraints

In [ ]:
// Meaningful: Generic repository with business constraints

public interface IEntity
{
    int Id { get; set; }
    DateTime CreatedDate { get; set; }
    DateTime? ModifiedDate { get; set; }
}

public interface ISoftDeletable
{
    bool IsDeleted { get; set; }
    DateTime? DeletedDate { get; set; }
}

public interface IAuditable
{
    string CreatedBy { get; set; }
    string ModifiedBy { get; set; }
}

// Generic repository with multiple meaningful constraints
public interface IRepository<T> where T : class, IEntity, new()
{
    Task<T> GetByIdAsync(int id);
    Task<IEnumerable<T>> GetAllAsync();
    Task<T> AddAsync(T entity);
    Task<T> UpdateAsync(T entity);
    Task<bool> DeleteAsync(int id);
}

// Specialized repository for soft-deletable entities
public interface ISoftDeleteRepository<T> : IRepository<T> 
    where T : class, IEntity, ISoftDeletable, new()
{
    Task<IEnumerable<T>> GetActiveAsync();
    Task<IEnumerable<T>> GetDeletedAsync();
    Task<bool> SoftDeleteAsync(int id);
    Task<bool> RestoreAsync(int id);
}

// Auditable repository with user context
public interface IAuditableRepository<T> : IRepository<T> 
    where T : class, IEntity, IAuditable, new()
{
    Task<T> AddAsync(T entity, string userId);
    Task<T> UpdateAsync(T entity, string userId);
    Task<IEnumerable<T>> GetByUserAsync(string userId);
}

### 🎯 Example 2: Data Processing Pipeline with Generic Constraints

In [ ]:
// Meaningful: Processing pipeline with type constraints for business logic

public interface IValidatable
{
    ValidationResult Validate();
}

public interface IProcessable
{
    ProcessingStatus Status { get; set; }
    DateTime ProcessedDate { get; set; }
}

// Generic processor with multiple constraints ensuring type safety
public class DataProcessor<TInput, TOutput> 
    where TInput : class, IValidatable, IProcessable
    where TOutput : class, new()
{
    private readonly ILogger<DataProcessor<TInput, TOutput>> _logger;
    private readonly List<Func<TInput, bool>> _validationRules;
    private readonly List<Func<TInput, TOutput>> _transformationRules;

    public DataProcessor(ILogger<DataProcessor<TInput, TOutput>> logger)
    {
        _logger = logger;
        _validationRules = new List<Func<TInput, bool>>();
        _transformationRules = new List<Func<TInput, TOutput>>();
    }

    // Generic method with business logic
    public async Task<ProcessingResult<TOutput>> ProcessAsync(
        IEnumerable<TInput> items, 
        CancellationToken cancellationToken = default)
    {
        var result = new ProcessingResult<TOutput>();
        var validItems = new List<TInput>();

        // Validate items using constraints
        foreach (var item in items)
        {
            var validationResult = item.Validate();
            if (validationResult.IsValid && PassesCustomValidation(item))
            {
                validItems.Add(item);
            }
            else
            {
                result.Errors.Add($"Validation failed for item: {validationResult.ErrorMessage}");
            }
        }

        // Process valid items
        foreach (var item in validItems)
        {
            try
            {
                var output = await TransformAsync(item, cancellationToken);
                result.ProcessedItems.Add(output);
                
                // Update processing status using constraint
                item.Status = ProcessingStatus.Completed;
                item.ProcessedDate = DateTime.UtcNow;
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Error processing item");
                result.Errors.Add($"Processing failed: {ex.Message}");
            }
        }

        return result;
    }

    // Generic method with constraints
    public void AddValidationRule<TRule>(Func<TInput, bool> rule) 
        where TRule : IValidationRule<TInput>
    {
        _validationRules.Add(rule);
    }

    // Generic method with transformation
    public void AddTransformationRule<TTransformer>(Func<TInput, TOutput> transformer)
        where TTransformer : ITransformer<TInput, TOutput>
    {
        _transformationRules.Add(transformer);
    }

    private bool PassesCustomValidation(TInput item)
    {
        return _validationRules.All(rule => rule(item));
    }

    private async Task<TOutput> TransformAsync(TInput input, CancellationToken cancellationToken)
    {
        var output = new TOutput();
        
        foreach (var transformer in _transformationRules)
        {
            output = transformer(input);
        }

        return output;
    }
}

public class ProcessingResult<T>
{
    public List<T> ProcessedItems { get; set; } = new();
    public List<string> Errors { get; set; } = new();
    public bool IsSuccessful => !Errors.Any();
    public int ProcessedCount => ProcessedItems.Count;
}

### 🎯 Example 3: Business Domain Collections with Constraints

In [ ]:
// Meaningful: Domain-specific collections with business rules

public interface IPriceable
{
    decimal Price { get; }
    string Currency { get; }
}

public interface IDiscountable
{
    decimal DiscountPercentage { get; set; }
    bool IsEligibleForDiscount { get; }
}

// Generic shopping cart with business constraints
public class ShoppingCart<TItem> 
    where TItem : class, IPriceable, IDiscountable
{
    private readonly List<CartItem<TItem>> _items;
    private readonly string _currency;
    private readonly decimal _maxCartValue;

    public ShoppingCart(string currency, decimal maxCartValue = 10000m)
    {
        _items = new List<CartItem<TItem>>();
        _currency = currency;
        _maxCartValue = maxCartValue;
    }

    // Generic method with business logic and constraints
    public bool TryAddItem<T>(T item, int quantity) where T : TItem
    {
        // Business rule: Currency must match
        if (item.Currency != _currency)
        {
            throw new InvalidOperationException($"Item currency {item.Currency} doesn't match cart currency {_currency}");
        }

        var potentialTotal = CalculateTotal() + (item.Price * quantity);
        
        // Business rule: Cart value limit
        if (potentialTotal > _maxCartValue)
        {
            return false;
        }

        var existingItem = _items.FirstOrDefault(i => i.Item.Equals(item));
        if (existingItem != null)
        {
            existingItem.Quantity += quantity;
        }
        else
        {
            _items.Add(new CartItem<TItem> { Item = item, Quantity = quantity });
        }

        return true;
    }

    // Generic method with business calculations
    public CartSummary<TItem> CalculateSummary<TDiscount>(TDiscount globalDiscount) 
        where TDiscount : IDiscount
    {
        var summary = new CartSummary<TItem>();
        summary.Currency = _currency;

        foreach (var cartItem in _items)
        {
            var itemTotal = cartItem.Item.Price * cartItem.Quantity;
            var itemDiscount = 0m;

            // Apply item-specific discount using constraint
            if (cartItem.Item.IsEligibleForDiscount)
            {
                itemDiscount = itemTotal * (cartItem.Item.DiscountPercentage / 100);
            }

            // Apply global discount using generic constraint
            var globalDiscountAmount = globalDiscount.CalculateDiscount(itemTotal - itemDiscount);

            summary.SubTotal += itemTotal;
            summary.TotalDiscount += itemDiscount + globalDiscountAmount;
        }

        summary.FinalTotal = summary.SubTotal - summary.TotalDiscount;
        return summary;
    }

    // Generic method with filtering
    public IEnumerable<T> GetItemsOfType<T>() where T : TItem
    {
        return _items.Where(i => i.Item is T).Select(i => (T)i.Item);
    }

    public decimal CalculateTotal()
    {
        return _items.Sum(i => i.Item.Price * i.Quantity);
    }

    public int ItemCount => _items.Sum(i => i.Quantity);
}

public class CartItem<T> where T : IPriceable
{
    public T Item { get; set; }
    public int Quantity { get; set; }
}

public class CartSummary<T> where T : IPriceable
{
    public decimal SubTotal { get; set; }
    public decimal TotalDiscount { get; set; }
    public decimal FinalTotal { get; set; }
    public string Currency { get; set; }
}

## ❌ Trivial Usage Examples (What to Avoid)

### 🚫 Example 1: Unnecessary Generic Wrapper

In [ ]:
// Trivial: Generic wrapper that doesn't add business value

public class GenericContainer<T>  // No constraints, no business logic
{
    public T Value { get; set; }  // Just a simple wrapper

    public GenericContainer(T value)
    {
        Value = value;  // No validation, no business rules
    }

    // Trivial method that could just be a property
    public T GetValue()
    {
        return Value;
    }

    // Trivial method with no business logic
    public void SetValue(T value)
    {
        Value = value;
    }
}

// Usage that doesn't benefit from generics
var stringContainer = new GenericContainer<string>("hello");
var intContainer = new GenericContainer<int>(42);
// This could just be regular variables: string value = "hello"; int number = 42;

### 🚫 Example 2: Generic Method Without Purpose

In [ ]:
// Trivial: Generic methods that don't need to be generic

public class TrivialService
{
    // Unnecessary generic method - could just return object
    public T DoNothing<T>(T input)  // No constraints, no logic
    {
        return input;  // Does nothing with the type
    }

    // Generic method that just calls ToString
    public string ConvertToString<T>(T value)
    {
        return value?.ToString() ?? string.Empty;  // Could be a simple method
    }

    // Generic method with no type safety benefits
    public void PrintValue<T>(T value)
    {
        Console.WriteLine(value);  // No constraint checking, no business logic
    }

    // Unnecessarily complex for simple operations
    public List<T> CreateList<T>(params T[] items)
    {
        return new List<T>(items);  // Built-in methods already exist
    }
}

// Usage that shows no benefit
var service = new TrivialService();
var result1 = service.DoNothing(42);  // Could just be: var result1 = 42;
var result2 = service.ConvertToString("hello");  // Could just be: "hello".ToString();

### 🚫 Example 3: Generic Class Without Constraints or Logic

In [ ]:
// Trivial: Generic class that doesn't leverage generic benefits

public class SimpleList<T>  // No constraints
{
    private List<T> _items = new List<T>();

    // Just delegates to List<T> without adding value
    public void Add(T item)
    {
        _items.Add(item);  // No validation, no business logic
    }

    public T Get(int index)
    {
        return _items[index];  // No bounds checking, no business rules
    }

    public int Count => _items.Count;  // Just a wrapper property
}

// This doesn't add any value over using List<T> directly
var list = new SimpleList<string>();  // Could just use List<string>
list.Add("item");  // No different from: new List<string>().Add("item");

## 💡 Key Principles for Meaningful Generic Usage

### ✅ What Makes Generic Usage Meaningful:

1. **Type Safety with Business Rules**
   - Constraints that enforce business requirements
   - Type checking at compile time for domain rules
   - Prevention of invalid type combinations

2. **Reusability Across Business Domains**
   - Generic components that work with multiple entity types
   - Repository patterns for different data models
   - Processing pipelines for various data types

3. **Performance Benefits**
   - Avoiding boxing/unboxing for value types
   - Type-specific optimizations
   - Reduced memory allocations

4. **Complex Constraint Combinations**
   - Multiple interface constraints for business logic
   - Constructor constraints for object creation
   - Reference/value type constraints when appropriate

### ❌ What Makes Generic Usage Trivial:

1. **Simple Wrappers**
   - Generic classes that just wrap a single value
   - No constraints or business logic
   - Could be replaced with regular variables

2. **Unnecessary Abstraction**
   - Generic methods that don't need to be generic
   - No type safety benefits
   - Just calling ToString() or basic operations

3. **Reinventing Built-in Functionality**
   - Creating generic collections that duplicate List<T>
   - Generic methods that duplicate existing framework methods
   - No added business value over existing solutions